<a href="https://colab.research.google.com/github/csce585-mlsystems/CSCE585ProjectROI/blob/michelleBranch/Model_Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



> Running experiments in ModelDevelopment Directory



Cloned the repository

In [ ]:
!git clone https://github.com/csce585-mlsystems/CSCE585ProjectROI

Cloning into 'CSCE585ProjectROI'...
remote: Enumerating objects: 526, done.
remote: Counting objects: 100% (222/222), done.
remote: Compressing objects: 100% (167/167), done.
remote: Total 526 (delta 90), reused 150 (delta 52), pack-reused 304 (from 1)
Receiving objects: 100% (526/526), 1.36 MiB | 10.15 MiB/s, done.
Resolving deltas: 100% (231/231), done.


After cloning, you might need to navigate into the repository's directory. Then, install any necessary Python packages. Often, these are listed in a `requirements.txt` file. You may need to check the repository's documentation for the exact file name or commands.

In [ ]:
# Repo directories
import os
os.chdir('CSCE585ProjectROI')


Here I'm uploading my dependencies

In [ ]:
!pip install reactpy reactpy-router flask llama-cpp-python TensorFlow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 MB 19.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.3/110.3 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 3.3 MB/s eta 0:00:00
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.3.16-cp312-cp312-linux_x86_64.whl size=4422346 sha256=376e4dd75e85b2c914f4f89cd62d2e0b222ccfe4b9e2b58da0dcbc25922ff98c
  Stored in directory: /root/.cache/pip/wheels/90/82/ab/8784ee3fb99ddb07fd36a679ddbe63122cc07718f6c1eb3be8
Successfully built llama-cpp-python


In [ ]:
import sys
print(sys.version)

3.12.12 (main, Oct 10 2025, 08:52:57) [GCC 11.4.0]


Verified that the notebook is running from python 3.12.12

Finally, you can load the model using the code provided by your classmate. The exact method will depend on how the model was saved (e.g., using `pickle`, `torch.load`, `tf.keras.models.load_model`). You will likely need to import specific libraries (like TensorFlow, PyTorch, or scikit-learn) depending on how the model was built.

Uploading Specific libaries

In [9]:
# Body of neccessary imports
import tensorflow as tf
import matplotlib
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np #<-- May be optional not sure as of 10/13/25.
# import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.utils import to_categorical, plot_model
from tensorflow.keras.datasets import mnist

Uploaded the model from local computer

Step 1: Split data into training group, validation and test sets

Step 2: Use data leakage prevention techniques such as walk forward cross validation

Step 3: Hypertune the parameters using the validation data set

Since we are measuring numerical values it would be best to use regression based evaluations such as MSE